In [19]:
#REFS:
#Short presentations for alternating and symmetric groups, J.N. Bray, M.D.E. Conder, C.R. Leedham-Green, and E.A. O’Brien
#https://en.wikipedia.org/wiki/Chinese_remainder_theorem#Generalization_to_arbitrary_rings

In [20]:
#we can represent the group algebra of a cyclic group as a polynomial ring quotient an ideal 
#F[C_N] = F[x]/(x^N-1)
#and factor x^N-1, and use the Chinese remainder theorem and Bezout's identity for the isomorphism.
#Similarly, we can write down a presentation of S_N, <\sigma, \tau | \sigma^n = 1, \tau^2 = 1, ...>,
#with just two generators and then the group algebra is a quotient of a free algebra on two generators,
#F[S_N] = F<x,y>/ (x^2 = y^n = (xy)^{n−1} = 1, (xy^{−1}xy)^3 = 1, (xy^{−j}xy^j)^2 = 1 for 2 \le j \le ⌊n/2⌋)
#We still have the Chinese remainder theorem, and there algorthims to factor non-commutative polynomials. 
#It turns out this is equivalent to using central orthogonal idempotents.
#Question: Do we gain anything with this approach?

In [21]:
#a short presentation for S_n
#{x,y|x^2 = y^n = (xy)^{n−1} = 1, (xy^{−1}xy)^3 = 1, (xy^{−j}xy^j)^2 = 1 for 2 \le j \le ⌊n/2⌋}
n=4
F.<a,b> = FreeGroup()
G = F / ([a^2, b^n, (a*b)^(n-1), (a*b^(-1)*a*b)^3] + [(a*b^(-j)*a*b^j)^2 for j in range(2,floor(n/2)+1)])
G.is_isomorphic(SymmetricGroup(n))

True

In [22]:
#however, the syemmtric group itself has the abilitity to solve the word problem
#so we just need an isomorphism G <--> S_n 
sym_n = SymmetricGroup(n); sym_n.gens()

((1,2,3,4), (1,2))

In [23]:
type(Hom(G,sym_n))

<class 'sage.groups.libgap_morphism.GroupHomset_libgap_with_category'>

In [24]:
#the group given by generators and relations does not have a built-in
g = G.an_element()
try:
    g.word_problem(G.gens())
except AttributeError as error:
    print(error)

'FinitelyPresentedGroup_with_category.element_class' object has no attribute 'word_problem'


In [25]:
g = sym_n.an_element()
g.word_problem(sym_n.gens())

x1*x2
[['(1,2,3,4)', 1], ['(1,2)', 1]]


('x1*x2', '(1,2,3,4)*(1,2)')

In [26]:
#g_algebra is not working properly with relations
A.<x,y> = FreeAlgebra(GF(3), 2)
try:
    P.<x,y> = A.g_algebra(relations={A(1):x^2},order = 'lex', check=False)
except ValueError as error:
    print(error)

not enough values to unpack (expected 2, got 0)


In [27]:
#Let I = I_1 \cap \ldots \cap I_k be the intersection of pairwise coprime two-sided ideals, 
#i.e. there exists i+j = 1 in each distinct pair I_i, I_j.
#Let phi: F<x,y>/I --> F<x,y>/I_1 \times \ldots \times F<x,y>/I_k
#Let f_i = (0,0, \ldots , 1 , \ldots 0, 0), a 1 in th i-th component and 0's elsewhere.
#The e_i = \phi^-1(f_i). I_i = F<x,y>(1-e_i).
#In this case we have the idempotents e_i = e_i(x,y), so we can construct the ideals I_i = F<x,y>(1-e_i). 
#So really we just need to compute 1-e_i(x,y), so convert the group elements their words in x, y.

In [28]:
#we can easily and quickly compute idempotents for the symmetric group algebra
p=3;
SGA_GFp_n = SymmetricGroupAlgebra(GF(p),n)
idems = SGA_GFp_n.central_orthogonal_idempotents()
#for each group element in idempotents, find expression as a word in x,y
list(idems[0])

[([1, 2, 4, 3], 2),
 ([1, 3, 2, 4], 2),
 ([1, 4, 3, 2], 2),
 ([2, 1, 3, 4], 2),
 ([3, 2, 1, 4], 2),
 ([4, 2, 3, 1], 2),
 ([2, 1, 4, 3], 1),
 ([3, 4, 1, 2], 1),
 ([4, 3, 2, 1], 1),
 ([2, 3, 4, 1], 1),
 ([2, 4, 1, 3], 1),
 ([3, 1, 4, 2], 1),
 ([3, 4, 2, 1], 1),
 ([4, 1, 2, 3], 1),
 ([4, 3, 1, 2], 1)]

In [29]:
SymmetricGroup(n)(list(idems[0])[0][0])

(3,4)

In [91]:
#can use free group algebra corresponding to g
R = GF(3)
FGA = GroupAlgebra(G, R); FGA

Algebra of Finitely presented group < a, b | a^2, b^4, (a*b)^3, (a*b^-1*a*b)^3, (a*b^-2*a*b^2)^2 > over Finite Field of size 3

In [122]:
#map from symmetric group algebra to quotient of free group algebra
#we must use the standard symmetric group in order to solve the word problem
#TO-DO: need to map solved word problem back to G
def SGA_to_FGA_quotient(v):
    #solve word problem for each basis element
    to_FGA_element = 0
    for pair in list(v):
        coeff = pair[1]
        sigma = SymmetricGroup(n)(pair[0])
        gens = SymmetricGroup(n).gens()
        word_gens, word_sym = sigma.word_problem(gens,display=False)
        #map word in generators to free group quotient
        word_to_free_group_quotient = sage_eval(word_gens.replace("x1","a").replace("x2","b"),locals={'a':FGA.0,'b':FGA.1})
        print(word_to_free_group_quotient)

In [123]:
SGA_to_FGA_quotient(idems[0])

ValueError: cannot invert self (= b)

In [124]:
FGA.0^-1

ValueError: cannot invert self (= a)